In [16]:
import torch
import torch.nn as nn
import scipy.io
import numpy as np
import json 
import os
import glob
import pandas as pd

In [15]:
"""Loads the 1DSfM Matlab data and converts it into the one required for the network """


for dirs in os.listdir("./MATLAB_SO3/datasets_matrices/"):
    name = dirs.split(".mat")[0]
    mat  = scipy.io.loadmat(os.path.join("./MATLAB_SO3/datasets_matrices/" , dirs))
    Z_incomplete = mat['Z_incomplete']
    Omega = mat["Omega"]
    # Z_complete = mat[Z_complete]
    data = []
    for i in range(0 , Z_incomplete.shape[0] , 3):
        for j in range(0 , Z_incomplete.shape[0] , 3):
            if(Omega[i][j]==1 ):
                data.append([i , j])
    gt_path = f"./datasets/mat-cmpl/{name}_gt.pt"
    torch.save(torch.from_numpy(Z_incomplete), gt_path)
    x = []
    y = []
    for idx in data:
        for i in range(3):
            for j in range(3):
                x.append(idx[0]+i)
                y.append(idx[1]+j)
    x = torch.tensor(x)
    y = torch.tensor(y)
    ys_ = Z_incomplete[x , y]
    ys_ = torch.tensor(ys_)
    obs_path = f"./datasets/mat-cmpl/{name}_obs.pt"
    torch.save([(x, y), ys_], obs_path)
    
    
    


In [3]:
""" Compiles results of all exps"""

import pandas as pd
import json
from pandas.io.json import json_normalize

dataframe  = []
idx =0 
for dir in glob.glob("./logs/*"):
    config_path = os.path.join(dir ,  "config.json")
    f =open(config_path)
    data = json.load(f)
    norm_data = pd.DataFrame(json_normalize(data))
    norm_data.insert(0 , 'Exp' , '')
    norm_data['Exp'] = idx
    dataframe.append(norm_data)
    idx+=1
df = pd.concat(dataframe)
df.to_csv("./results.csv")



ipykernel_launcher:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
